<a href="https://colab.research.google.com/github/ajinkyagh/Python_Practice/blob/main/Python_PySpark_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#EDA & Data Manipulation
#Creating a dataframe from List
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EDA Methods").getOrCreate()

data = [("Alice",25),("Bob",30), ("Charlie", None )]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()

+-------+----+
|   Name| Age|
+-------+----+
|  Alice|  25|
|    Bob|  30|
|Charlie|NULL|
+-------+----+



In [20]:
#read CSV file
from google.colab import drive
drive.mount('/content/gdrive')

df_customers = spark.read.csv('/content/gdrive/My Drive/Colab Notebooks/dataset/customers.csv', header=True, inferSchema=True)
df_customers.show()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
+-----------+-------+-----------+------------------+
|customer_id|   name|signup_date|             email|
+-----------+-------+-----------+------------------+
|          1|  Alice| 2021-01-01| user1@example.com|
|          2|    Bob| 2021-04-01| user2@example.com|
|          3|Charlie| 2021-06-30| user3@example.com|
|          4|  David| 2021-09-28|              NULL|
|          5|    Eva| 2021-12-27| user5@example.com|
|          6|  Frank| 2022-03-27| user6@example.com|
|          7|  Grace| 2022-06-25| user7@example.com|
|          8|  Helen| 2022-09-23| user8@example.com|
|          9|    Ian| 2022-12-22| user9@example.com|
|         10|   Jane| 2023-03-22|user10@example.com|
+-----------+-------+-----------+------------------+



In [21]:
#Read a parquet File into the a data frame
# df_orders = spark.read.parquet('/content/gdrive/My Drive/Colab Notebooks/dataset/orders.csv')

#Read a json File into the a data frame
#df = spark.read.json("path")

In [22]:
df.show(5)

+-------+----+
|   Name| Age|
+-------+----+
|  Alice|  25|
|    Bob|  30|
|Charlie|NULL|
+-------+----+



In [23]:
df.columns

['Name', 'Age']

In [24]:
df.count()

3

In [25]:
#Displays the schema
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



In [26]:
#Computes summary statistics for numerical columns.
df.describe().show()

+-------+-------+------------------+
|summary|   Name|               Age|
+-------+-------+------------------+
|  count|      3|                 2|
|   mean|   NULL|              27.5|
| stddev|   NULL|3.5355339059327378|
|    min|  Alice|                25|
|    max|Charlie|                30|
+-------+-------+------------------+



In [28]:
#Commutes column wise missing values.
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+----+---+
|Name|Age|
+----+---+
|   0|  1|
+----+---+

